In [35]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
import torch
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
import pandas as pd

#Start by loading in a bill training data which contains text and its summary:
bill = load_dataset("billsum", split="ca_test")

In [36]:
#Split the dataset into a 60 train and 40 test:
bill = bill.train_test_split(test_size=0.4)

In [37]:
#Load T5 tokenizer to process text and summary:
t5 = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(t5)

check = "summarize: "

In [38]:
#The preprocessing function:
#Check the prefix the input with a 'summarize: '.
#Truncate sequences to be the maximum length. I decided that 2000 should be the max input with 500 as the max output

def preprocess(examples):
    inputs = [check + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=2000, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=500, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [39]:
#To apply the preprocessing function over the entire dataset, use 🤗 Datasets map method.
tokenized_bill = bill.map(preprocess, batched=True)

Map:   0%|          | 0/742 [00:00<?, ? examples/s]

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

In [40]:
#Now create a batch of examples using DataCollatorForSeq2Seq.
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=t5)

In [41]:
#Then create a function that passes your predictions and labels to compute to calculate the ROUGE metric:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Note this is not used in the current model as we did not have enough time to fine tune the model

In [42]:
#Load T5 with AutoModelForSeq2SeqLM:
model = AutoModelForSeq2SeqLM.from_pretrained(t5)

In [49]:
#Text to be summarized:
full = "summarize: But don’t be fooled; your body is working hard when you’re in the pool. Water is denser than air, so moving through H2O puts more external pressure on your limbs than out-of-water training, studies have shown. Even better, that pressure is uniformly distributed. It doesn’t collect in your knees, hips or the other places that bear most of the burden when you exercise with gravity sitting on your shoulders. How you breathe during a swimming workout is another big differentiator, says David Tanner, a research associate at Indiana University and co-editor of an educational handbook on the science of swimming. During a run or bike ride, your breath tends to be shallow and your exhales forceful. “It’s the other way around with swimming,” says Tanner. “You breathe in quickly and deeply, and then let the air trickle out.” Because your head is under water when you swim, these breathing adjustments are vital, and they may improve the strength of your respiratory muscles, Tanner says. “This kind of breathing keeps the lung alveoli”—the millions of little balloon-like structures that inflate and deflate as your breathe—“from collapsing and sticking together.”."

In [52]:
#Test
summarizer = pipeline("summarization", model="stevhliu/my_awesome_billsum_model")
summarizer(full)

#Tokenize the text:
tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_billsum_model")
inputs = tokenizer(full, return_tensors="pt").input_ids

#Use generate()
model = AutoModelForSeq2SeqLM.from_pretrained("stevhliu/my_awesome_billsum_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

#Decode the generated token ids back into text:
real_out = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(real_out)

a swimming workout puts more external pressure on your limbs than out-of-water training, studies have shown. During a run or bike ride, your breath tends to be shallow and your exhales forceful. During a run or bike ride, your breath tends to be shallow and your exhales forceful.
